# User routes on the site
## Description
**Clickstream** is a sequence of user actions on a website. It allows you to understand how users interact with the site. In this task, you need to find the most frequent custom routes.

## Input data
Input data is а table with clickstream data in file `hdfs:/data/clickstream.csv`.

### Table structure
* `user_id (int)` - Unique user identifier.
* `session_id (int)` - Unique identifier for the user session. The user's session lasts until the identifier changes.
* `event_type (string)` - Event type from the list:
    * **page** - visit to the page
    * **event** - any action on the page
    * <b>&lt;custom&gt;</b> - string with any other type
* `event_type (string)` - Page on the site.
* `timestamp (int)` - Unix-timestamp of action.

### Browser errors
Errors can sometimes occur in the user's browser - after such an error appears, we can no longer trust the data of this session and all the following lines after the error or at the same time with it are considered corrupted and **should not be counted** in statistics.

When an error occurs on the page, a random string containing the word **error** will be written to the `event_type` field.

### Sample of user session
<pre>
+-------+----------+------------+----------+----------+
|user_id|session_id|  event_type|event_page| timestamp|
+-------+----------+------------+----------+----------+
|    562|       507|        page|      main|1620494781|
|    562|       507|       event|      main|1620494788|
|    562|       507|       event|      main|1620494798|
|    562|       507|        page|    family|1620494820|
|    562|       507|       event|    family|1620494828|
|    562|       507|        page|      main|1620494848|
|    562|       507|wNaxLlerrorU|      main|1620494865|
|    562|       507|       event|      main|1620494873|
|    562|       507|        page|      news|1620494875|
|    562|       507|        page|   tariffs|1620494876|
|    562|       507|       event|   tariffs|1620494884|
|    562|       514|        page|      main|1620728918|
|    562|       514|       event|      main|1620729174|
|    562|       514|        page|   archive|1620729674|
|    562|       514|        page|     bonus|1620729797|
|    562|       514|        page|   tariffs|1620731090|
|    562|       514|       event|   tariffs|1620731187|
+-------+----------+------------+----------+----------+
</pre>

#### Correct user routes for a given user:
* **Session 507**: main-family-main
* **Session 514**: main-archive-bonus-tariffs

Route elements are ordered by the time they appear in the clickstream, from earliest to latest.

The route must be accounted for completely before the end of the session or an error in the session.

## Task
You need to use the Spark SQL, Spark RDD and Spark DF interfaces to create a solution file, the lines of which contain **the 30 most frequent user routes** on the site.

Each line of the file should contain the `route` and `count` values **separated by tabs**, where:
* `route` - route on the site, consisting of pages separated by "-".
* `count` - the number of user sessions in which this route was.

The lines must be **ordered in descending order** of the `count` field.

## Criteria
You can get maximum of 3.5 points (final grade) for this assignment, depedning on the number of interface you manage to leverage. The criteria are as follows:

* 0.5 points – Spark SQL solution with 1 query
* 0.5 points – Spark SQL solution with <=2 queries
* 0.5 points – Spark RDD solution
* 0.5 points – Spark DF solution
* 0.5 points – your solution algorithm is relatively optimized, i.e.: no O^2 or O^3 complexities; appropriate object usage; no data leaks etc. This is evaluated by staff.
* 1 point – 1 on 1 screening session. During this session staff member can ask you questions regarding your solution logic, framework usage, questionable parts of your code etc. If your code is clean enough, the staff member can just ask you to solve a theoretical problem connected to Spark.


In [1]:
import findspark
findspark.init()

import pyspark
sc = pyspark.SparkContext(appName='sga_jupyter')

from pyspark.sql import SparkSession, Row
se = SparkSession(sc)

from pyspark.sql.context import SQLContext
sqlContext = SQLContext(sc)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2024-01-12 11:58:35,504 WARN yarn.Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as functions
from pyspark.sql.window import Window

In [3]:
! hadoop fs -copyFromLocal clickstream.csv

copyFromLocal: `clickstream.csv': File exists


In [4]:
sql_tab = sqlContext.read.option("delimiter", "\t").option("header", True).csv('clickstream.csv')
sql_tab.show(20)

+-------+----------+------------+----------+----------+
|user_id|session_id|  event_type|event_page| timestamp|
+-------+----------+------------+----------+----------+
|    562|       507|        page|      main|1695584127|
|    562|       507|       event|      main|1695584134|
|    562|       507|       event|      main|1695584144|
|    562|       507|       event|      main|1695584147|
|    562|       507|wNaxLlerrorU|      main|1695584154|
|    562|       507|       event|      main|1695584154|
|    562|       507|       event|      main|1695584154|
|    562|       507|       event|      main|1695584160|
|    562|       507|        page|    rabota|1695584166|
|    562|       507|       event|    rabota|1695584174|
|    562|       507|       event|    rabota|1695584181|
|    562|       507|       event|    rabota|1695584189|
|    562|       507|        page|      main|1695584194|
|    562|       507|       event|      main|1695584204|
|    562|       507|       event|      main|1695

# Spark SQL

This is one query which consists from the subquery:

In [5]:
#subquery

sql_tab_converted = sql_tab.withColumn("timestamp", sql_tab.timestamp.cast("int")) 

min_ts_df = sql_tab_converted \
    .select("user_id", "session_id", "timestamp") \
    .where(sql_tab_converted.event_type.contains("error")) \
    .groupBy('user_id', 'session_id').min('timestamp') \
    .withColumnRenamed('min(timestamp)', 'min_ts')


# query

sql_tab_converted \
    .join(min_ts_df, ['user_id', 'session_id'], 'left') \
    .where( (min_ts_df.min_ts.isNull()) | (sql_tab_converted.timestamp < min_ts_df.min_ts) ) \
    .where(sql_tab_converted.event_type == 'page') \
    .select('user_id', 'session_id', 'event_page') \
    .groupBy('user_id', 'session_id').agg(functions.collect_list(sql_tab_converted.event_page).alias('pages')) \
    .groupBy('user_id', 'session_id', 'pages') \
    .agg(functions.array_join('pages', '-').alias('route')) \
    .select('route') \
    .groupBy('route').count() \
    .orderBy(functions.desc('count')) \
    .show(30)

+--------------------+-----+
|               route|count|
+--------------------+-----+
|                main| 8090|
|        main-archive| 1091|
|         main-rabota| 1036|
|       main-internet|  879|
|          main-bonus|  865|
|           main-news|  760|
|        main-tariffs|  668|
|         main-online|  584|
|          main-vklad|  514|
| main-archive-rabota|  167|
| main-rabota-archive|  167|
|  main-bonus-archive|  139|
|   main-rabota-bonus|  137|
|    main-news-rabota|  134|
|   main-bonus-rabota|  133|
|main-archive-inte...|  131|
|    main-rabota-news|  129|
|main-internet-rabota|  128|
|   main-archive-news|  124|
|main-rabota-internet|  123|
|main-internet-arc...|  123|
|  main-archive-bonus|  117|
|main-tariffs-inte...|  114|
| main-internet-bonus|  113|
|   main-news-archive|  112|
|  main-news-internet|  108|
|main-archive-tariffs|  103|
|  main-internet-news|  101|
|main-tariffs-archive|  101|
|           main-main|   94|
+--------------------+-----+
only showing t

# Spark DataFrame

In [6]:
df = se.createDataFrame(sql_tab.rdd, sql_tab.schema)
df.show(20)

+-------+----------+------------+----------+----------+
|user_id|session_id|  event_type|event_page| timestamp|
+-------+----------+------------+----------+----------+
|    562|       507|        page|      main|1695584127|
|    562|       507|       event|      main|1695584134|
|    562|       507|       event|      main|1695584144|
|    562|       507|       event|      main|1695584147|
|    562|       507|wNaxLlerrorU|      main|1695584154|
|    562|       507|       event|      main|1695584154|
|    562|       507|       event|      main|1695584154|
|    562|       507|       event|      main|1695584160|
|    562|       507|        page|    rabota|1695584166|
|    562|       507|       event|    rabota|1695584174|
|    562|       507|       event|    rabota|1695584181|
|    562|       507|       event|    rabota|1695584189|
|    562|       507|        page|      main|1695584194|
|    562|       507|       event|      main|1695584204|
|    562|       507|       event|      main|1695

In [7]:
df_converted = df.withColumn("timestamp", df['timestamp'].cast("int"))

min_ts_df = df_converted \
    .select("user_id", "session_id", "timestamp") \
    .filter(df_converted.event_type.contains("error")) \
    .groupBy('user_id', 'session_id') \
    .agg(functions.min('timestamp').alias('min_ts'))

result_df = df_converted \
    .join(min_ts_df, ['user_id', 'session_id'], 'left') \
    .filter((min_ts_df.min_ts.isNull()) | (df_converted.timestamp < min_ts_df.min_ts)) \
    .filter(df_converted.event_type == 'page') \
    .select('user_id', 'session_id', 'event_page') \
    .groupBy('user_id', 'session_id') \
    .agg(functions.collect_list(df_converted.event_page).alias('pages')) \
    .groupBy('user_id', 'session_id', 'pages') \
    .agg(functions.array_join('pages', '-').alias('route')) \
    .select('route') \
    .groupBy('route') \
    .agg(functions.count('*').alias('count')) \
    .orderBy(functions.desc('count')) \
    .show(30)

+--------------------+-----+
|               route|count|
+--------------------+-----+
|                main| 8090|
|        main-archive| 1093|
|         main-rabota| 1037|
|       main-internet|  879|
|          main-bonus|  864|
|           main-news|  760|
|        main-tariffs|  669|
|         main-online|  584|
|          main-vklad|  514|
| main-archive-rabota|  167|
| main-rabota-archive|  167|
|  main-bonus-archive|  139|
|   main-rabota-bonus|  137|
|    main-news-rabota|  134|
|   main-bonus-rabota|  133|
|main-archive-inte...|  131|
|    main-rabota-news|  129|
|main-internet-rabota|  128|
|   main-archive-news|  125|
|main-rabota-internet|  123|
|main-internet-arc...|  123|
|  main-archive-bonus|  117|
|main-tariffs-inte...|  114|
| main-internet-bonus|  113|
|   main-news-archive|  112|
|  main-news-internet|  108|
|main-archive-tariffs|  103|
|  main-internet-news|  102|
|main-tariffs-archive|  102|
|           main-main|   94|
+--------------------+-----+
only showing t

# Spark RDD

In [8]:
rdd = sql_tab.rdd

We need to collect timestamp with errors if we want to separete events:

In [9]:
error_events = (rdd
    .filter(lambda line: 'error' in line['event_type'])
    .map(lambda line: (f"{line['user_id']}_{line['session_id']}", int(line['timestamp'])))
    .groupByKey()
    .map(lambda line:(line[0], min(line[1])))
    .collectAsMap()
)

In [10]:
list(error_events.items())[:10]

[('562_507', 1695584154),
 ('43_858', 1695588090),
 ('253_708', 1695594165),
 ('2089_676', 1695629584),
 ('2443_372', 1695643808),
 ('4816_114', 1695652430),
 ('2997_916', 1695656058),
 ('2513_71', 1695660463),
 ('2295_234', 1695666685),
 ('1104_783', 1695690545)]

In [11]:
errors_broadcast = sc.broadcast(error_events) # sent to cluster

Now we can apply our error_events to main data for filtering and make special output:

In [12]:
import math

filtered = (
    
rdd
    .map(lambda line: (f"{line['user_id']}_{line['session_id']}", [line['event_type'], line['event_page'], int(line['timestamp'])]))
    .filter(lambda line: line[0] not in errors_broadcast.value or line[1][2] < errors_broadcast.value[line[0]])
    .filter(lambda line: "page" == line[1][0])
    .map(lambda line : (line[0], line[1][1]))
    .groupByKey()
    .map(lambda line: (line[0], "-".join(list(line[1]))))
    .map(lambda line: (line[1], 1))
    .reduceByKey(lambda x, y: x+y)
    .sortBy(lambda line: line[1], ascending=False)
    .take(30)

)

In [13]:
filtered # result

[('main', 8090),
 ('main-archive', 1091),
 ('main-rabota', 1038),
 ('main-internet', 878),
 ('main-bonus', 864),
 ('main-news', 759),
 ('main-tariffs', 669),
 ('main-online', 584),
 ('main-vklad', 513),
 ('main-rabota-archive', 167),
 ('main-archive-rabota', 167),
 ('main-bonus-archive', 139),
 ('main-rabota-bonus', 137),
 ('main-news-rabota', 134),
 ('main-bonus-rabota', 133),
 ('main-archive-internet', 131),
 ('main-rabota-news', 129),
 ('main-internet-rabota', 128),
 ('main-archive-news', 125),
 ('main-internet-archive', 123),
 ('main-rabota-internet', 123),
 ('main-archive-bonus', 117),
 ('main-tariffs-internet', 114),
 ('main-internet-bonus', 114),
 ('main-news-archive', 112),
 ('main-news-internet', 108),
 ('main-archive-tariffs', 103),
 ('main-tariffs-archive', 102),
 ('main-internet-news', 102),
 ('main-main', 94)]

In [14]:
sc.stop()